In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [2]:
from random import random

from SourceCodeTools.models.training_config import get_config, save_config, load_config
from SourceCodeTools.code.data.dataset.Dataset import SourceGraphDataset, filter_dst_by_freq
from SourceCodeTools.models.graph.train.sampling_multitask2 import training_procedure, SamplingMultitaskTrainer
from SourceCodeTools.models.graph.train.objectives.NodeClassificationObjective import NodeClassifierObjective
from SourceCodeTools.models.graph.train.objectives.SubgraphClassifierObjective import SubgraphAbstractObjective, \
    SubgraphClassifierObjective, SubgraphEmbeddingObjective
from SourceCodeTools.models.graph.train.utils import get_name, get_model_base
from SourceCodeTools.models.graph import RGGAN
from SourceCodeTools.tabular.common import compact_property
from SourceCodeTools.code.data.file_utils import unpersist

import dgl
import torch
import numpy as np
from argparse import Namespace
from torch import nn
from datetime import datetime
from os.path import join
from functools import partial

Using backend: pytorch


# Prepare parameters and options

Full list of options that can be added can be found in `SourceCodeTools/models/training_options.py`. They are ment to be used as arguments for cli trainer. Trainer script can be found in `SourceCodeTools/scripts/train.py`.

There are a lot of parameters. Ones that might be of interest are marked with `***`.

In [3]:
tokenizer_path = "/home/ltv/Dropbox/sentencepiece_bpe.model"

data_path = "/home/ltv/Documents/source-graph/python/cubert_variable_misuse_subsample/with_ast"
subgraph_partition = join(data_path, "partition.json")
filecontent_path = join(data_path, "common_filecontent.json")

In [4]:
config = get_config(
    subgraph_id_column="file_id",
    data_path=data_path,
    model_output_dir=data_path,
    subgraph_partition=subgraph_partition,
    tokenizer_path=tokenizer_path,
    objectives="subgraph_clf"
)

In [5]:
config

{'DATASET': {'data_path': '/home/ltv/Documents/source-graph/python/cubert_variable_misuse_subsample/with_ast',
  'train_frac': 0.9,
  'filter_edges': None,
  'min_count_for_objectives': 5,
  'self_loops': False,
  'use_node_types': False,
  'use_edge_types': False,
  'no_global_edges': False,
  'remove_reverse': False,
  'custom_reverse': None,
  'restricted_id_pool': None,
  'random_seed': None,
  'subgraph_id_column': 'file_id',
  'subgraph_partition': '/home/ltv/Documents/source-graph/python/cubert_variable_misuse_subsample/with_ast/partition.json'},
 'TRAINING': {'model_output_dir': '/home/ltv/Documents/source-graph/python/cubert_variable_misuse_subsample/with_ast',
  'pretrained': None,
  'pretraining_phase': 0,
  'sampling_neighbourhood_size': 10,
  'neg_sampling_factor': 3,
  'use_layer_scheduling': False,
  'schedule_layers_every': 10,
  'elem_emb_size': 100,
  'embedding_table_size': 200000,
  'epochs': 100,
  'batch_size': 128,
  'learning_rate': 0.001,
  'objectives': 'subgr

# Create Dataset

In [6]:
dataset = SourceGraphDataset(
    **{**config["DATASET"], **config["TOKENIZER"]}
)

# Declare target loading function (labels)

In [7]:
def load_labels():
    filecontent = unpersist(filecontent_path)
    return filecontent[["id", "label"]].rename({"id": "src", "label": "dst"}, axis=1)

One or several objectives could be used

In [8]:
class Trainer(SamplingMultitaskTrainer):
    def create_objectives(self, dataset, tokenizer_path):
        self.objectives = nn.ModuleList()
        
        self.objectives.append(
            SubgraphClassifierObjective(
                self.graph_model, self.node_embedder, dataset.nodes,
                load_labels, self.device,
                self.sampling_neighbourhood_size, self.batch_size,
                tokenizer_path=tokenizer_path, target_emb_size=self.elem_emb_size, link_predictor_type="inner_prod",
                masker=None,
                measure_scores=self.trainer_params["measure_scores"],
                dilate_scores=self.trainer_params["dilate_scores"], nn_index=self.trainer_params["nn_index"],
                subgraph_mapping=dataset.subgraph_mapping,
                subgraph_partition=dataset.subgraph_partition
            )
        )

In [17]:
%tensorboard --logdir "/home/ltv/Documents/source-graph/python/cubert_variable_misuse_subsample/with_ast"

In [9]:
training_procedure(
    dataset, 
    model_name=RGGAN, 
    model_params=config["MODEL"],
    trainer_params=config["TRAINING"],
    model_base_path=get_model_base(config["TRAINING"], get_name(RGGAN, str(datetime.now()))),
    trainer=Trainer
)

Number of nodes 1014088


Epoch 1:   0%|                                                                                                                                                                             | 0/110 [00:00<?, ?it/s]

Epoch: 0, Time: 139 s
{'Accuracy/test/SubgraphClassifierObjective_': 0.5009635416666667,
 'Accuracy/train/SubgraphClassifierObjective_': 0.5714285714285714,
 'Accuracy/train_avg/SubgraphClassifierObjective': 0.5052658279220779,
 'Accuracy/val/SubgraphClassifierObjective_': 0.49108573717948717,
 'Loss/test/SubgraphClassifierObjective_': 0.6936271031697591,
 'Loss/train/SubgraphClassifierObjective_': 0.690739631652832,
 'Loss/train_avg/SubgraphClassifierObjective': 0.6951748934659091,
 'Loss/val/SubgraphClassifierObjective_': 0.6942669600248337}


Epoch 2:   0%|                                                                                                                                                                             | 0/110 [00:00<?, ?it/s]

Epoch: 1, Time: 139 s
{'Accuracy/test/SubgraphClassifierObjective_': 0.5008333333333334,
 'Accuracy/train/SubgraphClassifierObjective_': 0.5714285714285714,
 'Accuracy/train_avg/SubgraphClassifierObjective': 0.5091720779220779,
 'Accuracy/val/SubgraphClassifierObjective_': 0.49108573717948717,
 'Loss/test/SubgraphClassifierObjective_': 0.6932162642478943,
 'Loss/train/SubgraphClassifierObjective_': 0.691825270652771,
 'Loss/train_avg/SubgraphClassifierObjective': 0.693672450022264,
 'Loss/val/SubgraphClassifierObjective_': 0.6934565156698227}


Epoch 3:   0%|                                                                                                                                                                             | 0/110 [00:00<?, ?it/s]

Epoch: 2, Time: 146 s
{'Accuracy/test/SubgraphClassifierObjective_': 0.5009635416666667,
 'Accuracy/train/SubgraphClassifierObjective_': 0.5714285714285714,
 'Accuracy/train_avg/SubgraphClassifierObjective': 0.5090300324675324,
 'Accuracy/val/SubgraphClassifierObjective_': 0.49108573717948717,
 'Loss/test/SubgraphClassifierObjective_': 0.6933944970369339,
 'Loss/train/SubgraphClassifierObjective_': 0.6900871992111206,
 'Loss/train_avg/SubgraphClassifierObjective': 0.6938425871458921,
 'Loss/val/SubgraphClassifierObjective_': 0.6939537227153778}


Epoch 3:  55%|██████████████████████████████████████████████████████████████████████████████████████████▉                                                                         | 61/110 [00:52<00:42,  1.16it/s]


KeyboardInterrupt: 